In [1]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
outputs_path = os.getenv('J111_OUTPUTS')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Get SA2 report outputs
res_csv = os.path.join(outputs_path, 'OUT_Redland_SA2_ResGrowthAllocation.csv')
nonres_csv = os.path.join(outputs_path, 'OUT_Redland_SA2_NonResGrowthAllocation.csv')

# Get full path of Inputs GDB
input_gdb_path = os.path.join(spatial_path, 'redland_gam_current_dev.gdb')

# Set arcpy workspace
arcpy.env.workspace = os.path.join(env_path, 'J111_redland_gam.gdb')

# Import the SA2 boundaries feature class
areas = pd.DataFrame.spatial.from_featureclass(os.path.join(input_gdb_path, 'INPUT_FC_SA2_Redland'))

# Keep only the necessary fields
areas = areas.loc[:, ('sa2_name_2016', 'SHAPE')].rename({'sa2_name_2016' : 'sa2_name'}, axis=1).set_index('sa2_name')

# Inspect dataframe
areas.head()

,SHAPE
sa2_name,
Alexandra Hills,"{""rings"": [[[520884.7352, 6955803.8205], [5208..."
Birkdale,"{""rings"": [[[520949.8898, 6959402.5904], [5209..."
Capalaba,"{""rings"": [[[517452.33999999985, 6954562.2722]..."
Thorneside,"{""rings"": [[[519469.2489, 6960257.0201], [5195..."
Wellington Point,"{""rings"": [[[522621.9288999997, 6958480.4359],..."


In [2]:
# Join residential data
res_data = pd.read_csv(res_csv).set_index('sa2_name')
res_areas = areas.join(res_data).reset_index()
res_areas.head()

,sa2_name,SHAPE,det_dwl_2021,det_dwl_2026,det_dwl_2031,det_dwl_2036,det_dwl_2041,det_dwl_2046,det_dwl_2051,det_dwl_spare,det_dwl_cap,att_dwl_2021,att_dwl_2026,att_dwl_2031,att_dwl_2036,att_dwl_2041,att_dwl_2046,att_dwl_2051,att_dwl_spare,att_dwl_cap
0,Alexandra Hills,"{""rings"": [[[520884.7352, 6955803.8205], [5208...",5720.0,5763.0,5829.0,5859.0,5859.0,5859.0,5859.0,0.0,139.0,597.0,668.0,668.0,668.0,668.0,668.0,668.0,664.0,735.0
1,Birkdale,"{""rings"": [[[520949.8898, 6959402.5904], [5209...",4837.0,4880.0,4970.0,5117.0,5264.0,5296.0,5328.0,153.0,644.0,779.0,856.0,931.0,961.0,991.0,1004.0,1017.0,835.0,1073.0
2,Capalaba,"{""rings"": [[[517452.33999999985, 6954562.2722]...",5478.0,5507.0,5604.0,5718.0,5718.0,5718.0,5718.0,0.0,240.0,1628.0,2013.0,2423.0,2473.0,2523.0,2582.0,2642.0,3219.0,4279.0
3,Thorneside,"{""rings"": [[[519469.2489, 6960257.0201], [5195...",992.0,994.0,998.0,999.0,1000.0,1009.0,1018.0,26.0,52.0,597.0,596.0,596.0,596.0,596.0,596.0,596.0,349.0,385.0
4,Wellington Point,"{""rings"": [[[522621.9288999997, 6958480.4359],...",3884.0,4004.0,4101.0,4150.0,4199.0,4223.0,4247.0,110.0,473.0,559.0,684.0,684.0,684.0,684.0,684.0,684.0,958.0,1083.0


In [5]:
# Join non-residential data
uses = [
    'commercial',
    'community',
    'education',
    'health',
    'industrial',
    'other',
    'retail',
]
horizons = ['2021', '2026', '2031', '2036', '2041', '2046', '2051', 'spare', 'cap']

nonres_data = pd.read_csv(nonres_csv).set_index('sa2_name')

# Get the largest contributing industry by horizon
new_cols = pd.DataFrame(np.empty([len(nonres_data), len(horizons)], dtype=str), columns=[f'largest_{h}' for h in horizons])
nonres_data = pd.concat([nonres_data, new_cols], axis=1)

for i, r in nonres_data.iterrows():

    for h in horizons:

        v = 0
        uv = None

        for u in uses:

            if h == 'cap':

                tv = r[f'{u}_2051'] + r[f'{u}_spare']

            else:

                tv = r[f'{u}_{h}']

            if tv > v:

                v = tv
                uv = u

        nonres_data.at[i, f'largest_{h}'] = uv

# Join to spatial
nonres_areas = areas.join(nonres_data).reset_index()
nonres_areas.head()

,sa2_name,SHAPE,commercial_2021,commercial_2026,commercial_2031,commercial_2036,commercial_2041,commercial_2046,commercial_2051,commercial_spare,...,retail_cap,largest_2021,largest_2026,largest_2031,largest_2036,largest_2041,largest_2046,largest_2051,largest_spare,largest_cap
0,Alexandra Hills,"{""rings"": [[[520884.7352, 6955803.8205], [5208...",261.2190,261.219000,261.219000,261.219000,261.219000,261.219000,261.219000,1925.728400,...,7702.9136,education,education,education,education,education,education,education,education,education
1,Birkdale,"{""rings"": [[[520949.8898, 6959402.5904], [5209...",5080.6070,5080.607000,5441.973748,5701.070091,6032.558145,6427.493794,6427.493794,5705.387606,...,28209.0976,retail,retail,retail,retail,retail,retail,retail,retail,retail
2,Capalaba,"{""rings"": [[[517452.33999999985, 6954562.2722]...",96153.7045,128454.115419,149845.216696,174260.381988,198990.121009,223849.706123,248452.478876,78653.985024,...,515990.7806,retail,retail,retail,retail,retail,retail,retail,retail,retail
3,Thorneside,"{""rings"": [[[519469.2489, 6960257.0201], [5195...",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1860.830000,...,7443.3200,industrial,industrial,industrial,industrial,industrial,industrial,industrial,retail,industrial
4,Wellington Point,"{""rings"": [[[522621.9288999997, 6958480.4359],...",9639.4795,9639.479500,10051.792000,10051.792000,10051.792000,10051.792000,10051.792000,1874.113900,...,8854.4656,education,education,education,education,education,education,education,education,education


In [6]:
# Output
out_gdb = os.path.join(spatial_path, 'redland_gam_alloc.gdb')
res_areas.spatial.to_featureclass(os.path.join(out_gdb, 'OUT_FC_SA2_ResGrowthAllocation'), overwrite=True)
nonres_areas.spatial.to_featureclass(os.path.join(out_gdb, 'OUT_FC_SA2_NonResGrowthAllocation'), overwrite=True)

'G:\\Shared drives\\PIESolutions_03_Projects\\J000111 - Redlands planning assumption update\\06_Working Documents\\00_GIS Directory\\00_Data\\gam_model_run\\redland_gam_alloc.gdb\\OUT_FC_SA2_NonResGrowthAllocation'